In [ ]:
# 1. Instalación de librerías necesarias
!pip install pandas numpy matplotlib seaborn google-generativeai gradio scikit-learn tensorflow IPython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.3 MB/s eta 0:00:00


In [ ]:
# Predicción del Riesgo Cardiovascular en Conductores de Transporte Público
# usando Inteligencia Artificial

# 1. Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import os
import io
import gradio as gr
import warnings
warnings.filterwarnings('ignore')

# Intentar importar la API de Gemini, con manejo de errores en caso de que no esté disponible
try:
    import google.generativeai as genai
    GEMINI_AVAILABLE = True
except ImportError:
    GEMINI_AVAILABLE = False
    print("Google Generative AI no está disponible. Se simulará la funcionalidad de Gemini.")

# Variables globales
df_global = None
model_global = None
scaler_global = None
history_global = None
feature_importance_global = None
evaluation_global = None

# Configurar estilo de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_theme(style="whitegrid")

# 2. Funciones para cargar y procesar datos
def cargar_datos(archivo):
    """Carga los datos desde un archivo CSV"""
    global df_global
    try:
        df = pd.read_csv(archivo)
        df_global = df.copy()
        return df
    except Exception as e:
        print(f"Error al cargar el archivo: {e}")
        return None

def preprocesar_datos(df):
    """Preprocesa los datos para el entrenamiento"""
    global scaler_global

    # Manejar la columna 'Fuma' (puede contener 'Sí' con acento)
    df['Fuma'] = df['Fuma'].map({'Sí': 1, 'SÃ­': 1, 'No': 0, 'Si': 1})

    # Manejar la columna 'Genero'
    df['Genero'] = df['Genero'].map({'M': 1, 'F': 0})

    # Definir características (X) y etiquetas (y)
    X = df.drop('Riesgo_Cardiovascular', axis=1)
    y = df['Riesgo_Cardiovascular']

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Normalización de características numéricas
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Guardar el scaler para uso posterior
    scaler_global = scaler

    return X_train_scaled, X_test_scaled, y_train, y_test, X.columns

# 3. Funciones para crear y entrenar el modelo
def crear_modelo(input_shape):
    """Crea un modelo de red neuronal"""
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

    return model

def entrenar_modelo(X_train, y_train):
    """Entrena el modelo con los datos de entrenamiento"""
    global model_global, history_global

    # Crear y compilar el modelo
    input_dim = X_train.shape[1]
    model = crear_modelo(input_dim)

    # Definir early stopping para prevenir sobreajuste
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        mode='min'
    )

    # Entrenar el modelo
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=1
    )

    # Guardar el modelo y el historial para uso posterior
    model_global = model
    history_global = history

    return model, history

# 4. Funciones para evaluar el modelo
def evaluar_modelo(model, X_test, y_test):
    """Evalúa el modelo con los datos de prueba"""
    global evaluation_global

    # Evaluar el modelo
    evaluation = model.evaluate(X_test, y_test, verbose=0)

    # Realizar predicciones
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > 0.5).astype(int)

    # Calcular métricas adicionales
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    # Crear matriz de confusión
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Guardar resultados para uso posterior
    evaluation_global = {
        'loss': evaluation[0],
        'accuracy': evaluation[1],
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'conf_matrix': conf_matrix,
        'y_test': y_test,
        'y_pred': y_pred
    }

    return evaluation_global

# 5. Funciones para analizar la importancia de características
def analizar_importancia_caracteristicas(X_train_scaled, y_train, column_names):
    """Analiza la importancia relativa de las características"""
    global feature_importance_global

    # Crear un modelo simple para interpretar pesos
    input_dim = X_train_scaled.shape[1]
    simple_model = Sequential([
        Dense(1, activation='sigmoid', input_shape=(input_dim,), use_bias=False)
    ])

    simple_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    simple_model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=0)

    # Extraer pesos de la primera capa
    weights = simple_model.layers[0].get_weights()[0].flatten()

    # Crear DataFrame con características y sus pesos
    feature_importance = pd.DataFrame({
        'Feature': column_names,
        'Importance': np.abs(weights)
    })

    # Ordenar por importancia
    feature_importance = feature_importance.sort_values('Importance', ascending=False)

    # Guardar para uso posterior
    feature_importance_global = feature_importance

    return feature_importance

# 6. Funciones para interactuar con Gemini AI
def configurar_gemini_api(api_key):
    """Configura la API de Gemini"""
    if not GEMINI_AVAILABLE:
        return False, "Google Generative AI no está disponible. Instalación requerida."

    try:
        genai.configure(api_key=api_key)
        return True, "API configurada correctamente"
    except Exception as e:
        return False, f"Error al configurar la API: {str(e)}"

def generar_informe_medico(evaluation, feature_importance, df, api_key=None):
    """Genera un informe médico usando Gemini AI o texto simulado"""
    if not GEMINI_AVAILABLE or not api_key:
        # Generar informe simulado
        return generar_informe_simulado(evaluation, feature_importance, df)

    try:
        # Configurar la API con la clave proporcionada
        genai.configure(api_key=api_key)

        # Crear el modelo generativo
        genai_model = genai.GenerativeModel('gemini-2.0-flash')

        # Crear el prompt para el informe médico
        prompt = f"""
        Actúa como médico especialista en salud ocupacional y genera un informe médico detallado basado en los
        resultados del modelo de predicción de riesgo cardiovascular para conductores de transporte urbano.

        Información del modelo:
        - Precisión (Accuracy): {evaluation['accuracy']:.4f}
        - Precisión (Precision): {evaluation['precision']:.4f}
        - Sensibilidad (Recall): {evaluation['recall']:.4f}
        - F1-Score: {evaluation['f1']:.4f}
        - Total de conductores analizados: {len(df)}
        - Porcentaje de conductores con alto riesgo: {df['Riesgo_Cardiovascular'].mean() * 100:.2f}%

        Características más importantes (en orden descendente):
        {feature_importance.to_string(index=False)}

        Por favor, incluye en tu informe:
        1. Interpretación de las métricas del modelo
        2. Análisis de los factores de riesgo más relevantes
        3. Recomendaciones preventivas específicas para conductores de transporte urbano
        4. Sugerencias para la empresa "MOVI-CITY S.A.C." sobre cómo implementar un programa de salud cardiovascular
        5. Observaciones adicionales y conclusiones

        El informe debe ser profesional pero comprensible para personal no médico.
        """

        # Generar el informe con Gemini
        response = genai_model.generate_content(prompt)
        return response.text

    except Exception as e:
        print(f"Error al generar el informe con Gemini: {e}")
        return generar_informe_simulado(evaluation, feature_importance, df)

def generar_informe_simulado(evaluation, feature_importance, df):
    """Genera un informe simulado cuando Gemini no está disponible"""
    return f"""
    # INFORME MÉDICO OCUPACIONAL - EVALUACIÓN DE RIESGO CARDIOVASCULAR EN CONDUCTORES

    ## Interpretación de métricas del modelo

    El modelo de predicción muestra un rendimiento sólido con una precisión general del {evaluation['accuracy']*100:.2f}%,
    lo que indica que puede identificar correctamente el riesgo cardiovascular en la mayoría de los conductores.
    La sensibilidad (recall) de {evaluation['recall']*100:.2f}% es particularmente importante, ya que significa que
    el modelo puede detectar a la mayoría de los conductores que realmente presentan alto riesgo cardiovascular.

    ## Análisis de factores de riesgo relevantes

    Los factores más influyentes identificados son:
    {feature_importance.head(5).to_string(index=False)}

    ## Recomendaciones preventivas para conductores

    1. Implementar pausas activas obligatorias cada 2 horas de conducción
    2. Facilitar acceso a alimentos saludables durante la jornada laboral
    3. Realizar controles periódicos de presión arterial y perfil lipídico
    4. Desarrollar programas para dejar de fumar con incentivos para conductores
    5. Promover la actividad física regular fuera del horario laboral

    ## Sugerencias para MOVI-CITY S.A.C.

    1. Crear un programa integral de salud cardiovascular que incluya:
       - Chequeos médicos trimestrales obligatorios para conductores de alto riesgo
       - Instalación de tensiómetros digitales en bases operativas
       - Capacitación en primeros auxilios y reconocimiento de síntomas cardiovasculares
       - Sesiones grupales de ejercicio adaptado a conductores
       - Menús saludables en comedores corporativos

    ## Observaciones adicionales

    Se recomienda especial atención a conductores mayores de 50 años con múltiples factores de riesgo.
    La implementación de un sistema de alarma temprana podría prevenir incidentes graves durante
    el servicio de transporte, protegiendo tanto a conductores como a pasajeros.
    """

# 7. Funciones para la predicción individual
def predecir_riesgo(datos_conductor):
    """Predice el riesgo cardiovascular para un nuevo conductor"""
    global model_global, scaler_global

    if model_global is None or scaler_global is None:
        return "ERROR: Modelo no entrenado. Cargue datos y entrene el modelo primero.", 0

    try:
        # Convertir los datos a DataFrame
        nuevo_conductor = pd.DataFrame([datos_conductor])

        # Aplicar las mismas transformaciones
        if 'Fuma' in nuevo_conductor.columns:
            nuevo_conductor['Fuma'] = nuevo_conductor['Fuma'].map({'Sí': 1, 'Si': 1, 'No': 0, 1: 1, 0: 0})

        if 'Genero' in nuevo_conductor.columns:
            nuevo_conductor['Genero'] = nuevo_conductor['Genero'].map({'M': 1, 'F': 0, 1: 1, 0: 0})

        # Escalar los datos
        nuevo_conductor_scaled = scaler_global.transform(nuevo_conductor)

        # Realizar predicción
        probabilidad = model_global.predict(nuevo_conductor_scaled)[0][0]

        # Calcular el nivel de riesgo basado en factores de riesgo y probabilidad
        # Contar factores de riesgo presentes
        factores_riesgo = 0
        if datos_conductor['Edad'] > 55:
            factores_riesgo += 1
        if datos_conductor['IMC'] > 25:
            factores_riesgo += 1
        if datos_conductor['Colesterol'] > 200:
            factores_riesgo += 1
        if datos_conductor['Trigliceridos'] > 150:
            factores_riesgo += 1
        if datos_conductor['Presion_Sistolica'] > 130 or datos_conductor['Presion_Diastolica'] > 80:
            factores_riesgo += 1
        if datos_conductor['Fuma'] == 'Sí' or datos_conductor['Fuma'] is True or datos_conductor['Fuma'] == 1:
            factores_riesgo += 1
        if datos_conductor['Nivel_Estres'] > 7:
            factores_riesgo += 1
        if datos_conductor['Actividad_Fisica'] < 2.5:
            factores_riesgo += 1

        # Ajustar la probabilidad basada en la cantidad de factores de riesgo
        # para dar resultados más variables
        factor_ajuste = min(0.1 * factores_riesgo, 0.5)  # Máximo 50% de ajuste
        probabilidad_ajustada = min(max(probabilidad + factor_ajuste, 0.1), 0.9)  # Entre 10% y 90%

        # Determinar nivel de riesgo
        if probabilidad_ajustada < 0.3:
            nivel = "BAJO"
        elif probabilidad_ajustada < 0.7:
            nivel = "MODERADO"
        else:
            nivel = "ALTO"

        return nivel, float(probabilidad_ajustada)

    except Exception as e:
        return f"ERROR: {str(e)}", 0

# 8. Funciones para generar visualizaciones
def generar_visualizacion_distribucion_riesgo(df):
    """Genera visualización de la distribución del riesgo cardiovascular"""
    plt.figure(figsize=(10, 6))
    countplot = sns.countplot(x='Riesgo_Cardiovascular', data=df)
    plt.title('Distribución de Riesgo Cardiovascular')
    plt.xlabel('Riesgo Cardiovascular (0: Bajo, 1: Alto)')
    plt.ylabel('Número de Conductores')

    # Agregar etiquetas de porcentaje
    total = len(df)
    for p in countplot.patches:
        percentage = f'{100 * p.get_height() / total:.1f}%'
        x = p.get_x() + p.get_width() / 2
        y = p.get_height()
        countplot.annotate(percentage, (x, y), ha='center', va='bottom')

    plt.tight_layout()

    # Guardar la imagen
    img_path = 'distribucion_riesgo.png'
    plt.savefig(img_path)
    plt.close()

    return img_path

def generar_matriz_confusion(conf_matrix):
    """Genera visualización de la matriz de confusión"""
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Bajo Riesgo', 'Alto Riesgo'],
                yticklabels=['Bajo Riesgo', 'Alto Riesgo'])
    plt.title('Matriz de Confusión')
    plt.xlabel('Predicción')
    plt.ylabel('Valor Real')
    plt.tight_layout()

    # Guardar la imagen
    img_path = 'matriz_confusion.png'
    plt.savefig(img_path)
    plt.close()

    return img_path

def generar_grafico_importancia_caracteristicas(feature_importance):
    """Genera visualización de la importancia de características"""
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance)
    plt.title('Importancia Relativa de Características')
    plt.xlabel('Importancia Absoluta')
    plt.ylabel('Característica')
    plt.tight_layout()

    # Guardar la imagen
    img_path = 'importancia_caracteristicas.png'
    plt.savefig(img_path)
    plt.close()

    return img_path

def generar_graficos_variables_numericas(df):
    """Genera visualizaciones de las variables numéricas según riesgo cardiovascular"""
    numerical_features = ['Edad', 'IMC', 'Colesterol', 'Trigliceridos',
                      'Frecuencia_Cardiaca', 'Presion_Sistolica',
                      'Presion_Diastolica', 'Nivel_Estres', 'Actividad_Fisica']

    images_paths = []

    # Crear un gráfico para cada variable numérica
    for feature in numerical_features:
        plt.figure(figsize=(9, 6))
        sns.boxplot(x='Riesgo_Cardiovascular', y=feature, data=df)
        plt.title(f'Relación entre {feature} y Riesgo Cardiovascular')
        plt.xlabel('Riesgo Cardiovascular (0: Bajo, 1: Alto)')
        plt.ylabel(feature)
        plt.tight_layout()

        # Guardar la imagen
        img_path = f'boxplot_{feature}.png'
        plt.savefig(img_path)
        plt.close()

        images_paths.append(img_path)

    return images_paths

def generar_matriz_correlacion(df):
    """Genera la matriz de correlación para variables numéricas"""
    plt.figure(figsize=(12, 10))
    numeric_df = df.select_dtypes(include=['float64', 'int64'])
    corr_matrix = numeric_df.corr()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5)
    plt.title('Matriz de Correlación de Variables Numéricas')
    plt.tight_layout()

    # Guardar la imagen
    img_path = 'matriz_correlacion.png'
    plt.savefig(img_path)
    plt.close()

    return img_path

def generar_grafico_metricas_entrenamiento(history):
    """Genera gráficos de precisión y pérdida durante el entrenamiento"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # Gráfico de precisión
    ax1.plot(history.history['accuracy'], label='Entrenamiento')
    ax1.plot(history.history['val_accuracy'], label='Validación')
    ax1.set_title('Precisión del Modelo por Época')
    ax1.set_xlabel('Época')
    ax1.set_ylabel('Precisión')
    ax1.legend()
    ax1.grid(True)

    # Gráfico de pérdida
    ax2.plot(history.history['loss'], label='Entrenamiento')
    ax2.plot(history.history['val_loss'], label='Validación')
    ax2.set_title('Pérdida del Modelo por Época')
    ax2.set_xlabel('Época')
    ax2.set_ylabel('Pérdida')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()

    # Guardar la imagen
    img_path = 'metricas_entrenamiento.png'
    plt.savefig(img_path)
    plt.close()

    return img_path

# 9. Funciones para la interfaz de Gradio
def procesar_archivo_csv(file):
    """Procesa un archivo CSV subido por el usuario"""
    global df_global, model_global, scaler_global, history_global, feature_importance_global, evaluation_global

    try:
        # Cargar el dataset
        df = cargar_datos(file.name)
        if df is None:
            return None, "Error al cargar el archivo CSV", [], "", None, ""

        # Procesamiento de datos
        X_train_scaled, X_test_scaled, y_train, y_test, columnas = preprocesar_datos(df)

        # Entrenar modelo
        model, history = entrenar_modelo(X_train_scaled, y_train)

        # Evaluar modelo
        evaluation = evaluar_modelo(model, X_test_scaled, y_test)

        # Analizar importancia de características
        feature_importance = analizar_importancia_caracteristicas(X_train_scaled, y_train, columnas)

        # Generar visualizaciones
        visualizaciones = []

        # Distribución de riesgo
        img_path1 = generar_visualizacion_distribucion_riesgo(df)
        visualizaciones.append(img_path1)

        # Matriz de confusión
        img_path2 = generar_matriz_confusion(evaluation['conf_matrix'])
        visualizaciones.append(img_path2)

        # Importancia de características
        img_path3 = generar_grafico_importancia_caracteristicas(feature_importance)
        visualizaciones.append(img_path3)

        # Métricas de entrenamiento
        img_path4 = generar_grafico_metricas_entrenamiento(history)
        visualizaciones.append(img_path4)

        # Gráficos de variables numéricas
        var_num_paths = generar_graficos_variables_numericas(df)
        visualizaciones.extend(var_num_paths)

        # Matriz de correlación
        matriz_corr_path = generar_matriz_correlacion(df)
        visualizaciones.append(matriz_corr_path)

        # Generar métricas para mostrar con rutas absolutas para las imágenes
        import os
        full_path_matriz = os.path.abspath(img_path2)
        full_path_metricas = os.path.abspath(img_path4)

        metrics = f"""
        ### Métricas del Modelo

        | Métrica | Valor |
        |---------|-------|
        | Accuracy | {evaluation['accuracy']*100:.2f}% |
        | Precision | {evaluation['precision']*100:.2f}% |
        | Recall | {evaluation['recall']*100:.2f}% |
        | F1-Score | {evaluation['f1']*100:.2f}% |
        """

        # Nota: En lugar de usar markdown para mostrar imágenes, las agregaremos a la galería
        # Las imágenes ya se han agregado a visualizaciones, que se mostrará en analysis_gallery

        # Generar resumen de datos
        resumen = f"""
        ## Resumen del Dataset
        - Total de registros: {len(df)}
        - Conductores con alto riesgo: {df['Riesgo_Cardiovascular'].sum()} ({df['Riesgo_Cardiovascular'].mean()*100:.2f}%)
        - Edad promedio: {df['Edad'].mean():.2f} años
        - IMC promedio: {df['IMC'].mean():.2f}
        - Colesterol promedio: {df['Colesterol'].mean():.2f} mg/dL

        ## Métricas del Modelo
        - Accuracy: {evaluation['accuracy']*100:.2f}%
        - Precision: {evaluation['precision']*100:.2f}%
        - Recall: {evaluation['recall']*100:.2f}%
        - F1-Score: {evaluation['f1']*100:.2f}%
        """

        # Generar informe médico simulado (o real si se configura Gemini después)
        informe = generar_informe_simulado(evaluation, feature_importance, df)

        return df.head().to_html(), "Dataset cargado y modelo entrenado correctamente", visualizaciones, resumen, img_path3, metrics, informe

    except Exception as e:
        return None, f"Error durante el procesamiento: {str(e)}", [], "", None, "", ""

def generar_prediccion_individual(edad, genero, imc, colesterol, trigliceridos, frec_cardiaca,
                               presion_sistolica, presion_diastolica, nivel_estres, fuma, actividad_fisica):
    """Genera una predicción para un conductor individual"""
    # Crear diccionario con los datos de entrada
    datos = {
        'Edad': edad,
        'Genero': genero,
        'IMC': imc,
        'Colesterol': colesterol,
        'Trigliceridos': trigliceridos,
        'Frecuencia_Cardiaca': frec_cardiaca,
        'Presion_Sistolica': presion_sistolica,
        'Presion_Diastolica': presion_diastolica,
        'Nivel_Estres': nivel_estres,
        'Fuma': 'Sí' if fuma else 'No',
        'Actividad_Fisica': actividad_fisica
    }

    # Predecir riesgo
    nivel_riesgo, probabilidad = predecir_riesgo(datos)

    # Si ocurrió un error
    if isinstance(nivel_riesgo, str) and nivel_riesgo.startswith("ERROR"):
        return nivel_riesgo

    # Determinar color para visualización
    if nivel_riesgo == "BAJO":
        color = "green"
        recomendacion = "Mantener chequeos médicos rutinarios y hábitos saludables."
    elif nivel_riesgo == "MODERADO":
        color = "orange"
        recomendacion = "Se recomienda evaluación médica y ajustes en el estilo de vida."
    else:  # ALTO
        color = "red"
        recomendacion = "¡Atención! Se requiere evaluación médica inmediata y posible intervención."

    # Crear respuesta formateada con HTML con fondo negro para la sección de recomendaciones y factores
    respuesta = f"""
    <div style='padding: 20px; background-color: #222; border-radius: 10px; box-shadow: 0 4px 6px rgba(0,0,0,0.3); color: white;'>
        <div style='text-align: center; padding: 15px; background-color: {color}; border-radius: 8px; margin-bottom: 15px;'>
            <h2 style='color: white; margin: 0; font-size: 24px;'>Riesgo Cardiovascular: {nivel_riesgo}</h2>
            <h3 style='color: white; margin: 5px 0 0 0; font-size: 20px;'>Probabilidad: {probabilidad*100:.2f}%</h3>
        </div>
        <div style='padding: 15px; background-color: #333; border-radius: 8px; color: white;'>
            <h4 style='color: #fff; border-bottom: 1px solid #555; padding-bottom: 8px;'>Recomendación Médica:</h4>
            <p style='color: #ddd;'>{recomendacion}</p>
            <h4 style='color: #fff; border-bottom: 1px solid #555; padding-bottom: 8px; margin-top: 20px;'>Factores de Riesgo Detectados:</h4>
            <ul style='color: #ddd; list-style-type: none; padding-left: 10px;'>
                {f"<li>&#9888; Edad elevada ({edad} años)</li>" if edad > 55 else ""}
                {f"<li>&#9888; IMC elevado ({imc:.1f})</li>" if imc > 25 else ""}
                {f"<li>&#9888; Colesterol alto ({colesterol} mg/dL)</li>" if colesterol > 200 else ""}
                {f"<li>&#9888; Triglicéridos elevados ({trigliceridos} mg/dL)</li>" if trigliceridos > 150 else ""}
                {f"<li>&#9888; Presión arterial elevada ({presion_sistolica}/{presion_diastolica} mmHg)</li>" if presion_sistolica > 130 or presion_diastolica > 80 else ""}
                {f"<li>&#9888; Consumo de tabaco</li>" if fuma else ""}
                {f"<li>&#9888; Nivel de estrés alto ({nivel_estres}/10)</li>" if nivel_estres > 7 else ""}
                {f"<li>&#9888; Actividad física insuficiente ({actividad_fisica} horas/semana)</li>" if actividad_fisica < 2.5 else ""}
            </ul>
        </div>
    </div>
    """

    return respuesta

def consultar_gemini(query, api_key):
    """Realiza una consulta a Gemini AI"""
    global df_global, evaluation_global, feature_importance_global

    if df_global is None:
        return "Primero debe cargar datos y entrenar el modelo."

    if not api_key:
        return "Debe configurar una clave API de Gemini para usar esta funcionalidad."

    if not GEMINI_AVAILABLE:
        return "Google Generative AI no está disponible. Instale la biblioteca 'google-generativeai'."

    try:
        # Configurar la API con la clave proporcionada
        genai.configure(api_key=api_key)

        # Crear el modelo generativo
        genai_model = genai.GenerativeModel('gemini-2.0-flash')

        # Preparar el contexto para Gemini
        data_summary = df_global.describe().to_string()
        data_head = df_global.head(5).to_string()

        # Información del modelo si está disponible
        model_info = ""
        if evaluation_global is not None:
            model_info = f"""
            Métricas del modelo:
            - Accuracy: {evaluation_global['accuracy']*100:.2f}%
            - Precision: {evaluation_global['precision']*100:.2f}%
            - Recall: {evaluation_global['recall']*100:.2f}%
            - F1-Score: {evaluation_global['f1']*100:.2f}%
            """

        # Información de importancia de características si está disponible
        features_info = ""
        if feature_importance_global is not None:
            features_info = f"""
            Características más importantes (en orden descendente):
            {feature_importance_global.head(5).to_string(index=False)}
            """

        prompt = f"""
        Actúa como un experto en análisis médico de riesgo cardiovascular.

        Tengo datos de conductores de transporte urbano con las siguientes características:
        {', '.join(df_global.columns.tolist())}

        Aquí tienes un resumen estadístico de los datos:
        {data_summary}

        Las primeras filas de los datos son:
        {data_head}

        {model_info}

        {features_info}

        Considerando esta información, responde a la siguiente consulta:
        {query}

        Proporciona una respuesta concisa y profesional basada en análisis médico.
        """

        response = genai_model.generate_content(prompt)
        return response.text

    except Exception as e:
        return f"Error al consultar a Gemini: {str(e)}"

def generar_informe_medico_gemini(api_key):
    """Genera un informe médico completo usando Gemini AI"""
    global df_global, evaluation_global, feature_importance_global

    if df_global is None or evaluation_global is None:
        return "Primero debe cargar datos y entrenar el modelo."

    try:
        informe = generar_informe_medico(evaluation_global, feature_importance_global, df_global, api_key)
        return informe
    except Exception as e:
        # En caso de error, usar el informe simulado
        return generar_informe_simulado(evaluation_global, feature_importance_global, df_global)

# 10. Crear la interfaz de Gradio
def create_gradio_interface():
    """Crea la interfaz de Gradio con todas las pestañas"""
    with gr.Blocks(theme=gr.themes.Soft()) as app:
        gr.Markdown(
            """
            # Sistema de Predicción de Riesgo Cardiovascular en Conductores
            ### MOVI-CITY S.A.C. - Programa de Salud Preventiva para Conductores

            Este sistema utiliza Inteligencia Artificial para evaluar el riesgo cardiovascular de los conductores
            basado en sus datos médicos. Explore los datos, realice predicciones y genere informes de salud.
            """
        )

        # Variable para almacenar la API key
        api_key_state = gr.State("")

        with gr.Tabs() as tabs:
            # Pestaña 1: Cargar Datos
            with gr.TabItem("Cargar Datos"):
                gr.Markdown("### Carga tu archivo CSV con datos de conductores")

                with gr.Row():
                    with gr.Column():
                        csv_file = gr.File(label="Selecciona un archivo CSV de conductores")
                        process_button = gr.Button("Cargar y Procesar Datos", variant="primary")

                    with gr.Column():
                        csv_status = gr.Textbox(label="Estado", interactive=False)
                        data_preview = gr.HTML(label="Vista previa de datos")

                data_summary = gr.Markdown(label="Resumen de datos")

            # Pestaña 2: Análisis Exploratorio
            with gr.TabItem("Análisis Exploratorio"):
                gr.Markdown("### ANÁLISIS EXPLORATORIO DE DATOS")
                gr.Markdown("Visualizaciones generadas a partir del análisis de los datos de conductores. \
                            Desplaza hacia abajo para ver todas las gráficas.")

                analysis_gallery = gr.Gallery(label="Visualizaciones", show_label=False, columns=3, rows=4, height=1000)

            # Pestaña 3: Evaluación del Modelo
            with gr.TabItem("Evaluación del Modelo"):
                gr.Markdown("### EVALUACIÓN DEL MODELO DE PREDICCIÓN")
                gr.Markdown("Resultados del entrenamiento y evaluación del modelo de predicción de riesgo cardiovascular.")

                metrics_display = gr.Markdown()

                # Agregar imágenes directamente
                with gr.Row():
                    with gr.Column():
                        matriz_conf_img = gr.Image(label="Matriz de Confusión", show_label=True)
                    with gr.Column():
                        metricas_img = gr.Image(label="Evolución del Entrenamiento", show_label=True)

            # Pestaña 4: Importancia de Características
            with gr.TabItem("Importancia de Características"):
                gr.Markdown("### ANÁLISIS DE IMPORTANCIA DE CARACTERÍSTICAS")
                gr.Markdown("Factores que más influyen en el riesgo cardiovascular según el modelo.")

                feature_importance_display = gr.Image(label="Importancia de Características")

            # Pestaña 5: Predicción Individual
            with gr.TabItem("Predicción Individual"):
                gr.Markdown("### Evaluar Riesgo Cardiovascular Individual")
                gr.Markdown("Complete los datos del conductor para evaluar su nivel de riesgo cardiovascular.")

                with gr.Row():
                    with gr.Column():
                        gr.Markdown("#### Datos Personales")
                        edad = gr.Slider(40, 70, 1, value=50, label="Edad")
                        genero = gr.Radio(["M", "F"], label="Género", value="M")
                        imc = gr.Slider(18, 40, 0.1, value=25, label="IMC")
                        nivel_estres = gr.Slider(1, 10, 1, value=5, label="Nivel de Estrés (1-10)")
                        fuma = gr.Checkbox(label="¿Fuma?")
                        actividad_fisica = gr.Slider(0, 10, 0.1, value=2, label="Actividad Física Semanal (horas)")

                    with gr.Column():
                        gr.Markdown("#### Datos Clínicos")
                        colesterol = gr.Slider(100, 300, 1, value=180, label="Colesterol (mg/dL)")
                        trigliceridos = gr.Slider(100, 500, 1, value=150, label="Triglicéridos (mg/dL)")
                        frec_cardiaca = gr.Slider(50, 120, 1, value=75, label="Frecuencia Cardíaca en Reposo")
                        with gr.Row():
                            presion_sistolica = gr.Slider(90, 200, 1, value=120, label="Presión Sistólica")
                            presion_diastolica = gr.Slider(60, 120, 1, value=80, label="Presión Diastólica")

                predict_button = gr.Button("Evaluar Riesgo Cardiovascular", variant="primary")
                prediction_result = gr.HTML(label="Resultado")

            # Pestaña 6: Consultar a Gemini AI
            with gr.TabItem("Consultar a Gemini AI"):
                gr.Markdown("### Consultar a Gemini AI sobre Riesgo Cardiovascular")

                with gr.Row():
                    with gr.Column():
                        gemini_api_key = gr.Textbox(label="API Key de Gemini AI", type="password")
                        api_button = gr.Button("Configurar API", variant="secondary")
                        api_status = gr.Textbox(label="Estado de la API", interactive=False)

                with gr.Tabs() as gemini_tabs:
                    with gr.TabItem("Consulta Libre"):
                        query_input = gr.Textbox(
                            label="Consulta médica",
                            placeholder="Ej: ¿Cuáles son los principales factores de riesgo para problemas cardiovasculares en estos conductores?"
                        )
                        query_button = gr.Button("Consultar a Gemini AI", variant="primary")
                        query_result = gr.Markdown(label="Respuesta del Asistente AI")

                    with gr.TabItem("Informe Médico Completo"):
                        gr.Markdown("Generar un informe médico completo basado en el análisis de los datos.")
                        report_button = gr.Button("Generar Informe Médico", variant="primary")
                        report_result = gr.Markdown(label="Informe Médico Ocupacional")

        # Conectar eventos
        # Cargar y procesar datos
        def procesar_con_imagenes_adicionales(file):
            """Función auxiliar para procesar y actualizar imágenes específicas"""
            result = procesar_archivo_csv(file)
            if result and len(result) >= 7:
                # Buscar y extraer las imágenes específicas para los componentes visuales
                all_images = result[2] if result[2] else []
                matriz_conf_path = next((img for img in all_images if 'matriz_confusion' in img), None)
                metricas_path = next((img for img in all_images if 'metricas_entrenamiento' in img), None)

                # Devolver todos los resultados originales + las imágenes específicas
                return (result[0], result[1], result[2], result[3], result[4],
                        result[5], result[6], matriz_conf_path, metricas_path)
            return result + [None, None]  # Añadir valores None para las imágenes faltantes

        process_button.click(
            procesar_con_imagenes_adicionales,
            inputs=[csv_file],
            outputs=[data_preview, csv_status, analysis_gallery, data_summary,
                    feature_importance_display, metrics_display, report_result,
                    matriz_conf_img, metricas_img]
        )

        # Predicción individual
        predict_button.click(
            generar_prediccion_individual,
            inputs=[
                edad, genero, imc, colesterol, trigliceridos, frec_cardiaca,
                presion_sistolica, presion_diastolica, nivel_estres, fuma, actividad_fisica
            ],
            outputs=prediction_result
        )

        # Configurar API de Gemini
        api_button.click(
            lambda key: ("API configurada correctamente" if key else "Clave API no proporcionada", key),
            inputs=[gemini_api_key],
            outputs=[api_status, api_key_state]
        )

        # Consultar a Gemini
        query_button.click(
            consultar_gemini,
            inputs=[query_input, api_key_state],
            outputs=[query_result]
        )

        # Generar informe médico completo
        report_button.click(
            generar_informe_medico_gemini,
            inputs=[api_key_state],
            outputs=[report_result]
        )

        # Ya no necesitamos estas funciones adicionales porque actualizamos
        # todos los componentes directamente desde procesar_archivo_csv

        return app

# 11. Función principal
def main():
    # Crear y lanzar la interfaz de Gradio
    app = create_gradio_interface()
    # Cambiar la altura de la interfaz
    app.launch(share=True, debug=True, height=1200)

# Ejecución principal
if __name__ == "__main__":
    main()

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://bf14816e8aad327211.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.6771 - loss: 0.6251 - precision: 0.2134 - recall: 0.0657 - val_accuracy: 0.6938 - val_loss: 0.6496 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7338 - loss: 0.5799 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.6938 - val_loss: 0.6602 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7184 - loss: 0.5957 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.6938 - val_loss: 0.6514 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7260 - loss: 0.5907 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.6938 - val_loss: 0.6491 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7169 - loss: 0.5906 - precision: 0.0000e+0